In [165]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, accuracy_score, f1_score, roc_auc_score

In [166]:
data = pd.read_csv("/Users/trishanandakumar/Desktop/BURE/Datasets/loan_data.csv")

In [167]:
education_map = {'High School': 0, 'Associate': 1, 'Bachelor':2, 'Master':3, 'Doctorate':4}

data['person_education'] = data['person_education'].map(education_map)


hot_enc_cols = ['person_gender', 'person_home_ownership','loan_intent', 'previous_loan_defaults_on_file']
data = pd.get_dummies(data, columns=hot_enc_cols,drop_first=True)


to_fix_col = data.select_dtypes(include='bool').columns
data[to_fix_col] = data[to_fix_col].astype(int)


In [168]:
data.head(10)

,person_age,person_education,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status,person_gender_male,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,previous_loan_defaults_on_file_Yes
0,22.0,3,71948.0,0,35000.0,16.02,0.49,3.0,561,1,0,0,0,1,0,0,0,1,0,0
1,21.0,0,12282.0,0,1000.0,11.14,0.08,2.0,504,0,0,0,1,0,1,0,0,0,0,1
2,25.0,0,12438.0,3,5500.0,12.87,0.44,3.0,635,1,0,0,0,0,0,0,1,0,0,0
3,23.0,2,79753.0,0,35000.0,15.23,0.44,2.0,675,1,0,0,0,1,0,0,1,0,0,0
4,24.0,3,66135.0,1,35000.0,14.27,0.53,4.0,586,1,1,0,0,1,0,0,1,0,0,0
5,21.0,0,12951.0,0,2500.0,7.14,0.19,2.0,532,1,0,0,1,0,0,0,0,0,1,0
6,26.0,2,93471.0,1,35000.0,12.42,0.37,3.0,701,1,0,0,0,1,1,0,0,0,0,0
7,24.0,0,95550.0,5,35000.0,11.11,0.37,4.0,585,1,0,0,0,1,0,0,1,0,0,0
8,24.0,1,100684.0,3,35000.0,8.90,0.35,2.0,544,1,0,0,0,1,0,0,0,1,0,0
9,21.0,0,12739.0,0,1600.0,14.74,0.13,3.0,640,1,0,0,1,0,0,0,0,0,1,0


In [169]:
data.describe()
print(data.isnull().sum())

person_age                            0
person_education                      0
person_income                         0
person_emp_exp                        0
loan_amnt                             0
loan_int_rate                         0
loan_percent_income                   0
cb_person_cred_hist_length            0
credit_score                          0
loan_status                           0
person_gender_male                    0
person_home_ownership_OTHER           0
person_home_ownership_OWN             0
person_home_ownership_RENT            0
loan_intent_EDUCATION                 0
loan_intent_HOMEIMPROVEMENT           0
loan_intent_MEDICAL                   0
loan_intent_PERSONAL                  0
loan_intent_VENTURE                   0
previous_loan_defaults_on_file_Yes    0
dtype: int64


In [170]:
scaler = StandardScaler()

In [171]:
train_data, test_data = train_test_split(data, test_size=0.4, random_state=42, stratify=data['loan_status'])

In [172]:
rate_threshold = train_data['loan_int_rate'].quantile(0.93)
percent_income_threshold = train_data['loan_percent_income'].quantile(0.93)
loan_amnt_threshold = train_data['loan_amnt'].quantile(0.93) 



forget = train_data[(train_data['loan_int_rate'] >= rate_threshold) | 
              (train_data['loan_percent_income'] >= percent_income_threshold) |
              (train_data['loan_amnt'] >= loan_amnt_threshold)]

X_forget = forget.drop(columns=['loan_status'])
Y_forget = forget['loan_status']
len(forget)

5387

In [173]:
target = LogisticRegression(max_iter=1000)

X = train_data.drop('loan_status', axis=1)
Y =train_data['loan_status']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=42)

X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

target.fit(X_train_scale, Y_train)

LogisticRegression(max_iter=1000)

In [174]:
unlearn_scaler = StandardScaler()

In [175]:
keep = train_data.drop(forget.index)


unlearn = LogisticRegression(max_iter=1000)

X_unlearn = keep.drop('loan_status', axis=1)
Y_unlearn =keep['loan_status']


X_train_unlearn, X_test_unlearn, Y_train_unlearn, Y_test_unlearn = train_test_split(X_unlearn, Y_unlearn, test_size=.2, random_state=42)

X_train_unlearn_scale = unlearn_scaler.fit_transform(X_train_unlearn)
X_test_unlearn_scale = unlearn_scaler.transform(X_test_unlearn)

unlearn.fit(X_train_unlearn_scale , Y_train_unlearn)

LogisticRegression(max_iter=1000)

In [176]:
X_forget_scaled = unlearn_scaler.transform(X_forget)



predictions = target.predict(X_forget_scaled)
predictions_unlearn = unlearn.predict(X_forget_scaled)


print(f"Accuracy: {100*accuracy_score(Y_forget, predictions):.3f}")

print(f"F1: {100*f1_score(Y_forget, predictions, average='weighted'):.3f}")


print("\n")


print(f"Accuracy: {100*accuracy_score(Y_forget, predictions_unlearn):.3f}")

print(f"F1: {100*f1_score(Y_forget, predictions_unlearn, average='weighted'):.3f}")


print("Original model coef:", target.coef_)
print("Unlearned model coef:", unlearn.coef_)



Accuracy: 88.398
F1: 88.333


Accuracy: 83.423
F1: 83.316
Original model coef: [[ 6.63779980e-02 -4.62066727e-03  6.40252430e-02 -4.56865500e-02
  -6.27938334e-01  9.83351355e-01  1.35234426e+00 -3.95842759e-02
  -4.50136768e-01  3.51332529e-02  2.32302141e-02 -3.51229231e-01
   3.63238292e-01 -3.77655552e-01  2.56369415e-03 -9.68254614e-02
  -2.42110198e-01 -4.46091533e-01 -4.29789929e+00]]
Unlearned model coef: [[ 2.57384466e-01 -3.08240329e-03  2.43898520e-02 -1.85879841e-01
  -8.41654817e-01  7.71248431e-01  1.14881112e+00 -8.68932442e-02
  -4.22924374e-01 -4.64842182e-03 -1.31559635e-02 -3.43889124e-01
   1.84413604e-01 -2.76221018e-01  6.74101849e-02 -1.25489517e-01
  -1.68858850e-01 -4.25371405e-01 -3.95460995e+00]]


In [177]:
num_shadow_models = 5
chunks = int(len(test_data)/num_shadow_models)
shuffled_df = test_data.sample(frac=1, random_state=42).reset_index(drop=True)
shadow_sets = []


for num in range(num_shadow_models):
    start_index = int(chunks*num)
    end_index = int(start_index+chunks)

    shadow_train, shadow_test = train_test_split(shuffled_df[start_index:end_index], test_size=.5)
    shadow_sets.append((shadow_train, shadow_test))

print(len(shadow_sets[4][1]))

1800


In [178]:
shadow_models = []
shadow_scaled_data = []
shadow_scalers =[]
for num in range(num_shadow_models):
    shadow_scaler = StandardScaler()
    
    shadow_train = shadow_sets[num][0]
    X_shadow_train = shadow_train.drop('loan_status', axis=1)
    Y_shadow_train = shadow_train['loan_status']
    X_shadow_train_scaled = shadow_scaler.fit_transform(X_shadow_train)

    i = LogisticRegression(max_iter=1000)
    i.fit(X_shadow_train_scaled, Y_shadow_train )
    print("shadow model " + str(num) + " trained")

    shadow_scalers.append(shadow_scaler)
    shadow_scaled_data.append((X_shadow_train_scaled,Y_shadow_train))
    shadow_models.append((i))


shadow model 0 trained
shadow model 1 trained
shadow model 2 trained
shadow model 3 trained
shadow model 4 trained


In [179]:
member_feature= []
member_label = []

for num in range(num_shadow_models):

    model = shadow_models[num]
    scaled_data = shadow_scaled_data[num][0]
    Y_data = shadow_scaled_data[num][1]

    results = model.predict_proba(scaled_data)

    max_confidence = np.max(results, axis=1)
    entropy = -np.sum(results * np.log(results + 1e-8), axis=1)
    predictions = model.predict(scaled_data)
    true_class_proba = results[np.arange(len(Y_data)), Y_data]
    correctness = (predictions == Y_data).astype(int)
    loss = -np.log(true_class_proba + 1e-8)

    attack_features = np.column_stack([
        max_confidence,
        entropy, 
        true_class_proba,
        loss,
        correctness,
    ])
    
    member_feature.append(attack_features)
    member_label.append(np.ones(len(attack_features)))


In [180]:
non_member_feature= []
non_member_label = []

for num in range(num_shadow_models):

    model = shadow_models[num]
    test_set = shadow_sets[num][1]
    scaler = shadow_scalers[num]

    X_test = test_set.drop('loan_status', axis=1)
    Y_data_test = test_set['loan_status']
    X_test_scaled = scaler.fit_transform(X_test)

    results = model.predict_proba(X_test_scaled)


    max_confidence = np.max(results, axis=1)
    entropy = -np.sum(results * np.log(results + 1e-8), axis=1)
    predictions = model.predict(X_test_scaled)
    true_class_proba = results[np.arange(len(Y_data_test)), Y_data_test]
    correctness = (predictions == Y_data_test).astype(int)
    loss = -np.log(true_class_proba + 1e-8)

    attack_features = np.column_stack([
        max_confidence,
        entropy, 
        true_class_proba,
        loss,
        correctness,
    ])

    non_member_feature.append(attack_features)
    non_member_label.append(np.zeros(len(attack_features)))



In [181]:
X_attack = np.vstack(member_feature + non_member_feature)
y_attack = np.hstack(member_label + non_member_label)

In [182]:
attack_scaler = StandardScaler()

X_attack_scale = attack_scaler.fit_transform(X_attack)

attack = LogisticRegression(max_iter=1000)
attack.fit(X_attack_scale, y_attack)


LogisticRegression(max_iter=1000)

In [183]:
target_results = target.predict_proba(X_forget_scaled)

max_confidence = np.max(target_results, axis=1)
entropy = -np.sum(target_results * np.log(target_results + 1e-8), axis=1)
target_predictions = target.predict(X_forget_scaled)
true_class_proba = target_results[np.arange(len(Y_forget)), Y_forget]
correctness = (target_predictions == Y_forget).astype(int)
loss = -np.log(true_class_proba + 1e-8)

target_attack_features = np.column_stack([
        max_confidence,
        entropy, 
        true_class_proba,
        loss,
        correctness,
    ])

target_attack_features_scaled = attack_scaler.transform(target_attack_features)

In [184]:
unlearn_results = unlearn.predict_proba(X_forget_scaled)

max_confidence = np.max(unlearn_results, axis=1)
entropy = -np.sum(unlearn_results * np.log(unlearn_results + 1e-8), axis=1)
unlearn_predictions = unlearn.predict(X_forget_scaled)
true_class_proba = unlearn_results[np.arange(len(Y_forget)), Y_forget]
correctness = (unlearn_predictions == Y_forget).astype(int)
loss = -np.log(true_class_proba + 1e-8)

unlearn_attack_features = np.column_stack([
        max_confidence,
        entropy, 
        true_class_proba,
        loss,
        correctness,
    ])

unlearn_attack_features_scaled = attack_scaler.transform(unlearn_attack_features)

In [188]:
target_attack_predictions = attack.predict(target_attack_features_scaled)
target_attack_accuracy = accuracy_score(np.ones(len(target_attack_predictions)), target_attack_predictions)

In [189]:
unlearn_attack_predictions = attack.predict(unlearn_attack_features_scaled)
unlearn_attack_accuracy = accuracy_score(np.zeros(len(unlearn_attack_predictions)), unlearn_attack_predictions)

In [190]:
print(target_attack_accuracy)
print(unlearn_attack_accuracy)

0.8262483757193243
0.2949693707072582
